# Candlestick chart with Anchored VWAP

This notebook fetches OHLCV data from the local PostgreSQL `stock_data` table, resamples to the requested timeframe (if needed), computes an anchored VWAP starting from a user-provided anchor datetime, and plots an interactive candlestick chart with the anchored VWAP overlay and volume.

Inputs (change in the parameters cell):
- `ticker` (str) — symbol to plot
- `timeframe` (str) — pandas resample rule or bar size like `'1min'`, `'5min'`, `'15min'`, `'1H'`
- `anchor_datetime` (str) — ISO format datetime from which VWAP is anchored (e.g. `'2025-10-01 09:30:00'`)
- `start_date`, `end_date` (str) — window to query from DB; ensure anchor is inside window if you want VWAP drawn
- DB connection values (host, port, user, password, dbname)

Optional extras you may want later: timezone handling, smoothing of VWAP, multiple anchors, saving the plot as HTML.

In [28]:
# Parameters - edit these cells as needed
ticker = 'SPY'
timeframe = '1D'   # e.g. '1min', '5min', '15min', '1H'
anchor_datetime = '2025-04-06 00:00:00'  # ISO-like string; must be inside start/end window to show anchored VWAP
start_date = '2025-01-01 00:00:00'
end_date = '2025-10-27 23:59:59'
# DB connection (update if needed)
db_host = 'localhost'
db_port = 5432
db_user = 'khaled'
db_password = 'Arsenal4th-'
db_name = 'stockdata'
# Whether to show volume subplot
show_volume = True

In [9]:
# Install lightweight deps if running in a fresh environment (optional)
%pip install --quiet pandas sqlalchemy psycopg2-binary plotly

import pandas as pd
import sqlalchemy as sa
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import datetime
from IPython.display import display

# Utility: build SQLAlchemy engine
def get_engine(user, password, host, port, dbname):
    url = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'
    return sa.create_engine(url)

Note: you may need to restart the kernel to use updated packages.


In [16]:
def fetch_ohlcv_from_db(engine, ticker, start, end):
    q = sa.text('''
    SELECT datetime, open, high, low, close, volume
    FROM stock_data
    WHERE ticker = :ticker AND datetime BETWEEN :start AND :end
    ORDER BY datetime
    ''')
    # Use pandas read_sql with params to avoid injection/format issues
    df = pd.read_sql(q, engine, params={ 'ticker': ticker, 'start': start, 'end': end })
    # Ensure datetime column is datetime dtype and sorted
    if 'datetime' in df.columns:
        df['datetime'] = pd.to_datetime(df['datetime'])
        df = df.sort_values('datetime').set_index('datetime')
    else:
        raise ValueError('Query did not return a datetime column')
    return df

# Fetch and optionally resample into requested timeframe
def prepare_bars(df, timeframe):
    # Check if we already have the desired frequency
    current_freq = pd.infer_freq(df.index)
    
    # Map common timeframe strings to pandas frequencies for comparison
    freq_mapping = {
        '1min': 'T', '5min': '5T', '15min': '15T', '30min': '30T',
        '1H': 'H', '4H': '4H', '1D': 'D', '1W': 'W', '1M': 'M'
    }
    
    target_freq = freq_mapping.get(timeframe, timeframe)
    
    # If we already have the right frequency, return as-is
    if current_freq == target_freq:
        return df
    
    # Otherwise, resample to the requested timeframe
    rule = timeframe  # e.g. '1min', '5min' etc (pandas understands strings like '5min')
    agg = {
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum'
    }
    df_resampled = df.resample(rule).agg(agg).dropna(how='any')
    return df_resampled

In [ ]:
def compute_anchored_vwap(df, anchor_ts):
    # df must have columns: open, high, low, close, volume and datetime index
    # Ensure timezone consistency between df.index and anchor_ts
    if df.index.tz is not None and anchor_ts.tz is None:
        # If df.index is timezone-aware but anchor_ts is naive, localize anchor_ts
        anchor_ts = anchor_ts.tz_localize(df.index.tz)
    elif df.index.tz is None and anchor_ts.tz is not None:
        # If df.index is naive but anchor_ts is timezone-aware, make anchor_ts naive
        anchor_ts = anchor_ts.tz_localize(None)
    
    if anchor_ts not in df.index:
        # find next index >= anchor_ts to start from
        mask = df.index >= anchor_ts
        if not mask.any():
            raise ValueError('Anchor is outside available data range')
        start_idx = df.index[mask.argmax()]
    else:
        start_idx = anchor_ts

    # slice from anchor onwards
    df_anchor = df.loc[start_idx:].copy()
    
    # Debug: Print some sample data to understand the calculation
    print(f"Anchor point: {start_idx}")
    print(f"Sample data at anchor:")
    print(df_anchor.head(3)[['open', 'high', 'low', 'close', 'volume']])
    
    # typical price: (high+low+close)/3 is commonly used
    df_anchor['typ'] = (df_anchor['high'] + df_anchor['low'] + df_anchor['close']) / 3.0
    df_anchor['cum_typ_vol'] = (df_anchor['typ'] * df_anchor['volume']).cumsum()
    df_anchor['cum_vol'] = df_anchor['volume'].cumsum()
    df_anchor['anch_vwap'] = df_anchor['cum_typ_vol'] / df_anchor['cum_vol']
    
    # Debug: Print VWAP calculation details
    print(f"VWAP calculation sample:")
    print(df_anchor.head(5)[['close', 'typ', 'volume', 'cum_typ_vol', 'cum_vol', 'anch_vwap']])

    # Merge anchored vwap back into full df (NaN before anchor)
    df = df.join(df_anchor['anch_vwap'])
    return df, start_idx

In [27]:
# Run the pipeline and plot
engine = get_engine(db_user, db_password, db_host, db_port, db_name)
raw_df = fetch_ohlcv_from_db(engine, ticker, start_date, end_date)
print(f'Fetched {len(raw_df)} rows for {ticker}')
print(f'Raw data frequency: {pd.infer_freq(raw_df.index)}')
print(f'Raw data date range: {raw_df.index.min()} to {raw_df.index.max()}')
print(f'Raw data sample:')
print(raw_df.head())

bars = prepare_bars(raw_df, timeframe)
print(f'Resampled to {len(bars)} bars with timeframe {timeframe}')
print(f'Processed data frequency: {pd.infer_freq(bars.index)}')
print(f'Processed data sample:')
print(bars.head())

# compute anchored vwap
anchor_ts = pd.to_datetime(anchor_datetime)
# Ensure anchor_ts has same timezone as bars.index
if bars.index.tz is not None and anchor_ts.tz is None:
    anchor_ts = anchor_ts.tz_localize(bars.index.tz)
elif bars.index.tz is None and anchor_ts.tz is not None:
    anchor_ts = anchor_ts.tz_localize(None)

try:
    bars_with_vwap, used_anchor = compute_anchored_vwap(bars, anchor_ts)
    print(f'Anchored VWAP computed starting at {used_anchor}')
    print(f'VWAP sample values:')
    print(bars_with_vwap[['close', 'anch_vwap']].dropna().head())
except Exception as e:
    raise

# Plot using plotly: candlestick + anchored VWAP line, with optional volume subplot
fig = make_subplots(rows=(2 if show_volume else 1), cols=1, shared_xaxes=True,
                    vertical_spacing=0.03, row_heights=[0.7, 0.3] if show_volume else [1])

# Candlestick
fig.add_trace(go.Candlestick(x=bars_with_vwap.index,
                             open=bars_with_vwap['open'],
                             high=bars_with_vwap['high'],
                             low=bars_with_vwap['low'],
                             close=bars_with_vwap['close'],
                             name=f'{ticker}'), row=1, col=1)

# Anchored VWAP line (only non-null points)
fig.add_trace(go.Scatter(x=bars_with_vwap.index, y=bars_with_vwap['anch_vwap'], mode='lines', name='Anchored VWAP', line=dict(color='orange', width=2)), row=1, col=1)

if show_volume:
    fig.add_trace(go.Bar(x=bars_with_vwap.index, y=bars_with_vwap['volume'], name='Volume', marker_color='lightgray'), row=2, col=1)

fig.update_layout(title=f'{ticker} {timeframe} Candles with Anchored VWAP (anchor={used_anchor})', xaxis_rangeslider_visible=False, template='plotly_dark')
fig.update_xaxes(rangebreaks=[dict(bounds=['16:00','09:30'])])  # hide non-trading hours (customize as needed)
fig.show()

Fetched 78600 rows for NVDA
Raw data frequency: None
Raw data date range: 2025-01-02 14:30:00+00:00 to 2025-10-24 19:59:00+00:00
Raw data sample:
                             open    high     low   close   volume
datetime                                                          
2025-01-02 14:30:00+00:00  136.00  136.40  135.67  136.40  2584792
2025-01-02 14:31:00+00:00  136.39  137.68  136.38  137.55  1966079
2025-01-02 14:32:00+00:00  137.55  137.88  137.25  137.40  1406541
2025-01-02 14:33:00+00:00  137.42  137.74  136.70  136.74  1073426
2025-01-02 14:34:00+00:00  136.72  137.39  136.50  136.56   954712
Resampled to 202 bars with timeframe 1D
Processed data frequency: None
Processed data sample:
                             open    high     low   close     volume
datetime                                                            
2025-01-02 00:00:00+00:00  136.00  138.88  134.63  138.28  142568729
2025-01-03 00:00:00+00:00  140.00  144.90  139.73  144.59  163114333
2025-01-06 00:0

In [ ]:
# Manual VWAP verification
# Let's create a simple test case to verify our VWAP calculation
import pandas as pd
import numpy as np

# Create test data
test_data = pd.DataFrame({
    'close': [100, 105, 110, 108, 112],
    'high': [102, 107, 112, 110, 114],
    'low': [98, 103, 108, 106, 110],
    'volume': [1000, 2000, 1500, 3000, 2500]
})

test_data['typ'] = (test_data['high'] + test_data['low'] + test_data['close']) / 3.0
test_data['cum_typ_vol'] = (test_data['typ'] * test_data['volume']).cumsum()
test_data['cum_vol'] = test_data['volume'].cumsum()
test_data['vwap'] = test_data['cum_typ_vol'] / test_data['cum_vol']

print("Manual VWAP verification:")
print(test_data)

# Manual calculation for first 3 periods:
# Period 1: typ=100, vol=1000, cum_typ_vol=100000, cum_vol=1000, vwap=100
# Period 2: typ=105, vol=2000, cum_typ_vol=100000+210000=310000, cum_vol=3000, vwap=103.33
# Period 3: typ=110, vol=1500, cum_typ_vol=310000+165000=475000, cum_vol=4500, vwap=105.56

print("\nExpected VWAP values:")
print("Period 1: 100.00")
print("Period 2: 103.33") 
print("Period 3: 105.56")


Notes:
- If your DB stores timestamps without timezone, make sure the anchor_datetime and start/end use the same timezone or are naive in the same way.
- The resample rule is flexible; use pandas offset strings (e.g. '2min', '15min', '1H').
- You can easily extend this to compute VWAP anchored at multiple anchors by repeating `compute_anchored_vwap` for each anchor and adding multiple traces.